# Challenge 6: Actions
## Introduction

In this part of the challenge you will add another agent to the solution.

This time will be an agent that performs actions on behalf of the user.

### Import the necessary libraries

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries

import requests
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from urllib.parse import quote_plus 
from its_a_rag import ingestion
from sqlalchemy import create_engine
from langchain.agents import AgentExecutor, create_sql_agent, create_openai_tools_agent
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.tools import tool

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [3]:
import os, dotenv, sys
dotenv.load_dotenv()

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Local Folder for the documents
LOCAL_FOLDER = "D:\fsi2023"

# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DRIVER = os.getenv("SQL_DRIVER")

# STOCK API
MOCK_API_ENDPOINT= os.getenv("MOCK_API_ENDPOINT")

## Step 1: Build on top of the previous solution

In [4]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(api_key = AZURE_OPENAI_API_KEY,  
                      api_version = "2024-06-01",
                      azure_endpoint = AZURE_OPENAI_ENDPOINT,
                      model= AZURE_OPENAI_DEPLOYMENT_NAME,
                      streaming=True)

In [5]:
# Create the Agent State Class to store the messages between the agents
class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [6]:
# Stock Agent

def stock_agent(state):
    # Import the LLM
    global llm
    stock_agent_llm = llm
    # Create the DB Connection
    connection_string = f"Driver={SQL_DRIVER};Server=tcp:{SQL_SERVER},1433;Database={SQL_DB};Uid={SQL_USERNAME};Pwd={SQL_PWD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
    quoted_conn_str = quote_plus(connection_string)
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted_conn_str))
    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    db = SQLDatabase(engine=engine)
    stock_toolkit = SQLDatabaseToolkit(db=db, llm=stock_agent_llm)
    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    stock_agent = create_sql_agent(
        toolkit=stock_toolkit,
        llm=stock_agent_llm,
        agent_type="openai-tools",
        agent_name="StockAgent",
        agent_description="Stock Agent",
        agent_version="0.1",
        agent_author="itsarag",
        #verbose=True,
        agent_executor_kwargs=dict(handle_parsing_errors=True))
    # Structure the final prompt from the ChatPromptTemplate
    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", 
            """
            You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices. \n
            If you can't find the answer, say 'I am unable to find the answer.'
            """
            ),
            ("user", "{question}\n ai: "),
        ]
    )
    # Prepare the response using the invoke method of the agent
    response = stock_agent.invoke(final_prompt.format(question=state["input"]))
    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {"output": response['output'], "input": state["input"]}

In [7]:
## Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)

def rag_agent(state):
    # Define the LLM
    global llm
    rag_agent_llm = llm
    # Define the index (use the one created in the previous challenge - as the index is huge, we are using a top_k of 10)
    retriever_multimodal = AzureAISearchRetriever(index_name=AZURE_SEARCH_INDEX, api_key=AZURE_SEARCH_API_KEY, service_name=AZURE_SEARCH_ENDPOINT, top_k=10)
    # Define the chain (as it was in the previous challenge)
    chain_multimodal_rag = (
    {
        "context": retriever_multimodal | RunnableLambda(ingestion.get_image_description),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(ingestion.multimodal_prompt)
    | rag_agent_llm
    | StrOutputParser()
    )
    # prepare the response using the invoke method of the agent
    response = chain_multimodal_rag.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response, "input": state["input"]}

### Step 2: Create the Stock Action Agent

In [57]:
## Node stock_action
# Define the tool using the @tool decorator
@tool
def buy_stocks(quantity: int, symbol: str) -> str:
    '''Permit to acquire/buy a certain quantity of stocks of a certain company identified by its symbol'''
    if not symbol or not quantity:
        return f"Missing required arguments: symbol and quantity are required."
    try:
        response = requests.post(f"{MOCK_API_ENDPOINT}/api/buy/?stock_symbol={symbol}&quantity={quantity}")
        response.raise_for_status()
        return response.json()["message"]
    except requests.exceptions.RequestException as e:
        return f"Failed to buy stock. Exception: {str(e)}"


@tool
def sell_stocks(quantity: int, symbol: str) -> str:
    '''Permit to sell a certain quantity of stocks of a certain company identified by its symbol'''
    if not symbol or not quantity:
        return f"Missing required arguments: symbol and quantity are required."
    try:
        response = requests.post(f"{MOCK_API_ENDPOINT}/api/sell/?stock_symbol={symbol}&quantity={quantity}")
        response.raise_for_status()
        return response.json()["message"]
    except requests.exceptions.RequestException as e:
        return f"Failed to sell stock. Exception: {str(e)}"


# you can combine multiple tools in a single block called Toolkit
stock_toolkit = [buy_stocks, sell_stocks]

def stock_action(state):
    # Define the LLM
    global llm
    stock_action_llm = llm
    # Prepare the prompt for the agent (the create_openai_tools_agent function requires a ChatPromptTemplate object)
    # Prompt Example: "You are an agent that helps to acquire stocks. Use your tools to perform the requested action. \n if you can't perform the action, say 'I am unable to perform the action.' \n"
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", 
            """
            You are an agent that helps to acquire stocks. Use your tools to perform the requested action. \n
            If you can't perform the action, say 'I am unable to perform the action.'
            """
            ),
            MessagesPlaceholder("chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )
    # Construct the OpenAI Tools Agent
    stock_buyer = create_openai_tools_agent(stock_action_llm, stock_toolkit, prompt)
    # Prepare the Agent Executor
    stock_buyer_executor = AgentExecutor(agent=stock_buyer, tools=stock_toolkit)
    # prepare the response using the invoke method of the agent
    response = stock_buyer_executor.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response['output'], "input": state["input"]}
    
    

### Step 3: Update the Start Agent adding an addition decision answer

In [60]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent(state):
    # Import the global llm
    global llm
    start_agent_llm = llm
    # Prepare the prompt for the agent
    prompt = PromptTemplate.from_template("""
    You are an agent that needs analyze the user question. \n
    Question : {input} \n
    if the question is related to stock prices answer with "stock". \n
    if the question is related to stock sbuy/sell answer with "stock_action". \n
    if the question is related to information about financial results answer with "rag". \n
    if the question is unclear or you cannot decide answer with "rag". \n
    only answer with one of the word provided.
    Your answer (stock/sotck_action/rag):
    """)
    # Prepare the chain to be executed
    chain = prompt | start_agent_llm
    # invoke the chain
    response = chain.invoke({"input": state["input"]})
    # take the decision from the response
    decision = response.content.strip().lower()
    # Return the response for the next agent (decision and input required coming fron the Agent State)
    return {"decision": decision, "input": state["input"]}

### Step 4: Update the previous graph to include the new agent

In [61]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str

# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    workflow = StateGraph(AgentState)
    # Add the nodes (start_agent, stock_agent, rag_agent)
    workflow.add_node("start", start_agent)
    workflow.add_node("stock_agent", stock_agent)
    workflow.add_node("rag_agent", rag_agent)
    workflow.add_node("stock_action", stock_action)
    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent
    workflow.add_conditional_edges(
        "start",
        lambda x: x["decision"],
        {
            "stock": "stock_agent",
            "rag": "rag_agent",
            "stock_action": "stock_action"
        }
    )
    # Set the workflow entry point
    workflow.set_entry_point("start")
    # Add the final edges to the END node
    workflow.add_edge("stock_agent", END)
    workflow.add_edge("rag_agent", END)
    workflow.add_edge("stock_action", END)
    #Compile the workflow
    return workflow.compile()

### Step 5: Test the Solution

In [ ]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions

# Question 1
question = "What was the price of Apple,Nvidia and Microsoft stocks in 23/07/2024?"
result = graph.invoke({"input": question})
print(result["output"])

# Question 2
question = "Can you buy for me 100 stocks of MSFT?"
result = graph.invoke({"input": question})
print(result["output"])

# Question 3
question = "What is the value of the cumulative 5-years total return of Alphabet Class A stock at December 2022?"
result = graph.invoke({"input": question})
print(result["output"])